In [191]:
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [192]:
num=1
fn = ['DRB동일.csv', 'KEC.csv', 'SK아이이테크놀로지.csv', '기신정기.csv', '동양피스톤.csv', '두산퓨얼셀.csv', '디와이파워.csv', '삼아알미늄.csv', '티와이홀딩스.csv', '화승코퍼레이션.csv']

df = pd.read_csv('data/aside/'+fn[num], index_col = 'Date', parse_dates=True)
df['Mid']=(df['Low']+df['High'])/2

cut_line = len(df['Mid'])-round(len(df['Mid'])/10)

train_data = df['Mid'].values[:cut_line].reshape(-1,1).astype('float')
test_data = df['Mid'].values[cut_line:].reshape(-1,1).astype('float')


from sklearn.preprocessing import minmax_scale as mm

train_data = mm(train_data)
test_data = mm(test_data)

In [193]:
def create_seq(data, seq_length):
    x = []
    y = []
    for i in range(len(data)-seq_length):
        x.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    x = np.array(x)
    y = np.array(y)
    return x, y

In [194]:
x_train, y_train = create_seq(train_data, 7)
x_test, y_test = create_seq(test_data, 7)

In [195]:
x_train, y_train, x_test, y_test = map(lambda data:torch.from_numpy(data).float(), [x_train, y_train, x_test, y_test])

In [196]:
y_train[:2]

tensor([[0.6802],
        [0.6828]])

In [197]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device

In [198]:
class LSTM1(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, seq_len):
        super(LSTM1, self).__init__()
        self.num_layers = num_layers #number of layers 
        self.input_size = input_size #input size (number of features)
        self.hidden_size = hidden_size #hidden state 
        self.seq_len = seq_len #sequence length 
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True) #lstm 
        self.fc = nn.Linear(self.hidden_size, 1) #fully connected last layer 

    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, self.seq_len, self.hidden_size)).to(device) #hidden state 
        c_0 = Variable(torch.zeros(self.num_layers, self.seq_len, self.hidden_size)).to(device) #internal state 
        # Propagate input through LSTM 

        lstm_out, (hn, cn) = self.lstm(x.view(len(x), self.seq_len, -1), (h_0, c_0)) #lstm with input, hidden, and internal state 
        lstm_last = lstm_out.view(self.seq_len, len(x), self.hidden_size)[-1]
        out = self.fc(lstm_last)

        return out

In [199]:
num_epochs = 5000 #seems to converge at 10000 for 화송코퍼레이션
learning_rate = 0.001

input_size = 1 #number of features
hidden_size = 2 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers
seq_len = 7

lstm1 = LSTM1(input_size, hidden_size, num_layers, seq_len).to(device)

loss_function = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate)  # adam optimizer


In [201]:
for epoch in range(num_epochs):
    y_pred = lstm1.forward(x_train)  
    loss = loss_function(outputs, y_train) 
    # obtain the loss function 
    optimizer.zero_grad() 
    loss.backward() #calculates the loss of the loss function 
    optimizer.step() #improve from loss, i.e backprop 
    if epoch % 100 == 0: 
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

RuntimeError: Expected hidden[0] size (1, 3342, 2), got [1, 7, 2]

In [181]:
# plt.rc('font', family='NanumGothic') # use when korean text output is needed

df_X_ss = ss.transform(X)
df_X_ss = Variable(torch.Tensor(df_X_ss))

#reshaping the dataset
df_X_ss = torch.reshape(df_X_ss, (df_X_ss.shape[0], 1, df_X_ss.shape[1]))
data_predict = lstm1(df_X_ss.to(device)).data.detach().cpu().numpy()
data_predict = mm.inverse_transform(data_predict) #reverse transformation

history_cut = 0 # length of latest history to view. Set to 0 to see whole test set.

plt.figure(figsize=(7.5,4.5)) #plotting
plt.plot(y.values[cut_line:][-history_cut:], label='actual price', marker='o') #actual plot
plt.plot(data_predict[cut_line:][-history_cut:], label='predicted price', marker='v') #predicted plot
plt.title("Company I".split(".")[0])
plt.legend(loc='upper left')
plt.show() 

NameError: name 'ss' is not defined

In [189]:
import math

inp = inpuut.split("\n")
test = int(inp[0])

lit=[]

for i in range(test):
    p = [int(x) for x in inp[i+1].split()]
    print(p)
    x1=p[0]
    y1=p[1]
    r1=p[2]
    x2=p[3]
    y2=p[4]
    r2=p[5]
    
    d=math.sqrt((x1-x2)**2+(y1-y2)**2)
    if abs(r1-r2)!=d or r1+r2>d: lit.append(0)
    elif abs(r1-r2)==d or r1+r2==d: lit.append(1)
    else: lit.append(2)
    
for i in lit:
    print(i)

[0, 0, 13, 40, 0, 37]
[0, 0, 3, 0, 7, 4]
[1, 1, 1, 1, 1, 5]
0
0
0


In [186]:
inpuut='''3
0 0 13 40 0 37
0 0 3 0 7 4
1 1 1 1 1 5'''